In [1]:
#Import Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [2]:
#Read in csv
df = pd.read_csv ('/Users/tanushrisingh/Desktop/Spring 2019/Big Data Management/Assignment4/digit-recognizer/train.csv')
print(df.shape)

(42000, 785)


In [3]:
y=df['label'].values
x=df.drop(['label'],1).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)
print(y_test.shape)

(6300,)


In [4]:
batch_size = 32
torch_x_trn = torch.from_numpy(x_train).type(torch.LongTensor)
torch_y_trn = torch.from_numpy(y_train).type(torch.LongTensor)

#Creating features/targets
torch_x_tst = torch.from_numpy(x_test).type(torch.LongTensor)
torch_y_tst = torch.from_numpy(y_test).type(torch.LongTensor)

#Perform traning and testing
trn = torch.utils.data.TensorDataset(torch_x_trn, torch_y_trn)
tst = torch.utils.data.TensorDataset(torch_x_tst, torch_y_tst)

#Load into data
train_load = torch.utils.data.DataLoader(trn, batch_size=batch_size, shuffle=False)
test_load = torch.utils.data.DataLoader(tst, batch_size=batch_size, shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1=nn.Linear(784,250)
        self.linear2=nn.Linear(250,100)
        self.linear3=nn.Linear(100,10)
        
    def forward(self,x):
        x=F.relu(self.linear1(x))
        x=F.relu(self.linear2(x))
        x=self.linear3(x)
        return F.log_softmax(x,dim=1)
    
net = Net()
print(net)

Net(
  (linear1): Linear(in_features=784, out_features=250, bias=True)
  (linear2): Linear(in_features=250, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)


In [6]:
def fitModel(model, train_load):
    optimizer = torch.optim.Adam(model.parameters())
    err = nn.CrossEntropyLoss()
    epochs = 5
    model.train()
    for epoch in range(epochs):
        correct = 0
        for batchIndex, (x_batch, y_batch) in enumerate(train_load):
            var_x_batch = Variable(x_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_x_batch)
            loss = err(output, var_y_batch)
            loss.backward()
            optimizer.step()
            
            predict = torch.max(output.data, 1)[1]
            correct = (predict == var_y_batch).sum() + correct
            
            if batchIndex%50 == 0:
               print('Epoch : {}\t Accuracy:{:.3f}%'.format(epoch, float(correct*100) / float(batch_size*(batchIndex+1))))
            

In [ ]:
fitModel(net,train_load)

Epoch : 0	 Accuracy:6.250%
Epoch : 0	 Accuracy:65.993%
Epoch : 0	 Accuracy:74.938%
Epoch : 0	 Accuracy:78.974%
Epoch : 0	 Accuracy:81.297%
Epoch : 0	 Accuracy:83.155%
Epoch : 0	 Accuracy:84.417%
Epoch : 0	 Accuracy:85.301%
Epoch : 0	 Accuracy:86.004%
Epoch : 0	 Accuracy:86.648%
Epoch : 0	 Accuracy:87.088%
Epoch : 0	 Accuracy:87.528%
Epoch : 0	 Accuracy:87.916%
Epoch : 0	 Accuracy:88.201%
Epoch : 0	 Accuracy:88.583%
Epoch : 0	 Accuracy:88.902%
Epoch : 0	 Accuracy:89.217%
Epoch : 0	 Accuracy:89.472%
Epoch : 0	 Accuracy:89.720%
Epoch : 0	 Accuracy:89.859%
Epoch : 0	 Accuracy:90.069%
Epoch : 0	 Accuracy:90.271%
Epoch : 0	 Accuracy:90.460%
Epoch : 1	 Accuracy:96.875%
Epoch : 1	 Accuracy:94.179%
Epoch : 1	 Accuracy:94.183%
Epoch : 1	 Accuracy:94.123%
Epoch : 1	 Accuracy:94.108%
Epoch : 1	 Accuracy:94.447%
Epoch : 1	 Accuracy:94.414%
Epoch : 1	 Accuracy:94.560%
Epoch : 1	 Accuracy:94.599%
Epoch : 1	 Accuracy:94.741%
Epoch : 1	 Accuracy:94.711%
Epoch : 1	 Accuracy:94.725%
Epoch : 1	 Accuracy:9

In [ ]:
def evaluate(model):
#model = net
    correct = 0 
    for test_imgs, test_labels in test_load:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy -> {:.3f}% ".format( float(correct) / (len(test_load)*batch_size)))
evaluate(net)